# **Clustering des titres**

Le but de ce notebook est d'essayer de faire un clustering sur la colonne titres de notre fichier, afin de pouvoir ensuite donner un thème par cluster 

In [39]:
# Importation des bibliothèques nécessaires

import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
import numpy as np
from nltk.corpus import stopwords
import nltk
import spacy
from nltk.corpus import wordnet
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# Télécharger les stop words français si nécessaire
nltk.download('stopwords', quiet=True)
french_stop_words = stopwords.words('french')
english_stop_words = stopwords.words('english')

In [40]:
# Chargement des données 
data= pd.read_csv("articles.csv")
# Nombre d'articles 
print (data.shape[0])
print (data.columns)
# Remplacer le nom de la première colonne par "id"
data.rename(columns={data.columns[0]: "id"}, inplace=True)
print (data.columns)

1269
Index(['id_article', 'title', 'year', 'language', 'Nb_authors', 'Nb_male',
       'Nb_female', 'theme', 'keywords'],
      dtype='object')
Index(['id', 'title', 'year', 'language', 'Nb_authors', 'Nb_male', 'Nb_female',
       'theme', 'keywords'],
      dtype='object')


In [41]:
# Dans le fichier y'a des articles en français et en anglais 
data_fr = data[data['language'] == 'fr']
print (data_fr.shape[0])
data_en = data[data['language'] == 'en']
print (data_en.shape[0])

1133
122


# Traitement du français

In [42]:
!python -m spacy download fr_core_news_sm

     ---------------------------------------- 0.0/16.3 MB ? eta -:--:--
      --------------------------------------- 0.3/16.3 MB ? eta -:--:--
     --- ------------------------------------ 1.3/16.3 MB 4.8 MB/s eta 0:00:04
     -------- ------------------------------- 3.4/16.3 MB 6.9 MB/s eta 0:00:02
     ------------ --------------------------- 5.0/16.3 MB 7.0 MB/s eta 0:00:02
     --------------- ------------------------ 6.3/16.3 MB 6.9 MB/s eta 0:00:02
     ------------------- -------------------- 7.9/16.3 MB 6.9 MB/s eta 0:00:02
     ----------------------- ---------------- 9.7/16.3 MB 7.1 MB/s eta 0:00:01
     ---------------------------- ----------- 11.5/16.3 MB 7.4 MB/s eta 0:00:01
     -------------------------------- ------- 13.1/16.3 MB 7.4 MB/s eta 0:00:01
     ------------------------------------ --- 14.7/16.3 MB 7.4 MB/s eta 0:00:01
     ---------------------------------------- 16.3/16.3 MB 7.5 MB/s  0:00:02
✔ Download and installation successful
You can now load the packa

In [43]:
# Récupération du vocabulaire français à partir des titres des articles

import spacy
from spacy.lang.fr.stop_words import STOP_WORDS as french_stop_words

# Charger le modèle français
nlp = spacy.load("fr_core_news_sm")

# Initialiser un set vide pour le vocabulaire
vocab_fr = set()

for title in data_fr['title']:
    doc = nlp(title)
    for token in doc:
        # garder seulement les mots alphabétiques, minuscules, non stopwords
        lemma = token.lemma_.lower()
        if token.is_alpha and lemma not in french_stop_words:
            vocab_fr.add(lemma)

print("Nombre de mots uniques :", len(vocab_fr))


Nombre de mots uniques : 1924


In [44]:
print(vocab_fr)

{'visualisation', 'perforecast', 'perte', 'partage', 'modularité', 'exploration', 'registre', 'teximus', 'k', 'composer', 'création', 'traçabilité', 'sweetdeki', 'outlier', 'bâtiment', 'discrétiser', 'catégorisation', 'accident', 'lien', 'repérage', 'scientifique', 'temps', 'reuters', 'caracteristique', 'bruter', 'distribution', 'supplémentaire', 'stabilité', 'moyenner', 'web', 'diagramme', 'limite', 'méthode', 'date', 'vidéo', 'méthodologie', 'polluer', 'méta', 'tableau', 'puce', 'convexe', 'poids', 'ascendante', 'knowledge', 'csp', 'beaucoup', 'ras', 'tbi', 'allocataire', 'growth', 'ontologie', 'évolution', 'exceptionnel', 'divergente', 'owl', 'selection', 'economique', 'manipulation', 'elaboration', 'modélisation', 'bicluster', 'associative', 'centrer', 'clustering', 'agentuml', 'assainissement', 'conception', 'récit', 'solubilité', 'unique', 'egc', 'recommandation', 'simulation', 'influenceur', 'personnalité', 'thème', 'volume', 'substitution', 'thermique', 'annoter', 'binaire', 'p

In [45]:
# Nettoyage manuel des mots non pertinents
manual_stop_words = {
    # Bruit & Lettres isolées
    'p', 'n', 'b', 'c', 'j', 'v', 'm', 'r', 'k', 'cl', 'sou', 'tr', 'vs', 'via', 'multi', 'intro', 'chapitre',
    
    # Termes génériques de recherche
    'approche', 'approches', 'méthode', 'méthodes', 'système', 'systèmes',
    'application', 'applications', 'analyse', 'analyses', 'étude', 'études',
    'modèle', 'modèles', 'modélisation', 'algorithme', 'algorithmes',
    'problème', 'problèmes', 'solution', 'solutions', 'résultat', 'résultats',
    'outil', 'outils', 'processus', 'technique', 'techniques', 'concept',
    'contexte', 'cadre', 'travail', 'travaux',
    'contribution', 'contributions', 'cas', 'exemple', 'exemples', 'usage',
    'utilisation', 'proposition', 'démarche', 'principe', 'théorie', 'théorique',
    'pratique', 'état', 'art', 'vue', 'niveau', 'type', 'moyen', 'question',
    'enjeu', 'domaine', 'sujet', 
    
    # Verbes courants (formes lemmatisées probables)
    'utiliser', 'permettre', 'baser', 'proposer', 'présenter','définir', 'construire', 'générer', 'traiter',
    'extraire', 'apprendre',  'considérer', 'mesurer', 'voir',
    'faire', 'obtenir','viser', 'fournir', 'intégrer',
    'mettre', 'partir',  'montrer', 'conclure',
    
    # Adjectifs / Adverbes vagues
    'nouveau', 'nouvelle', 'nouveaux', 'nouvelles', 'bon', 'meilleur',
    'grand', 'petit', 'simple',
    'général', 'efficace', 'performant', 'rapide',  'classique', 'possible',
    'nécessaire', 'principal', 'important', 'haut', 'faible', 'large',
    'court', 'long', 'très', 'trop', 'peu', 'bien', 'mieux'
}
vocab_fr = vocab_fr - manual_stop_words
print(len(vocab_fr))

1837


In [46]:
# Créons une colonne titre_clean en les mettant en miniscule sans stop word et en lemme 
def clean_title_fr(text):
    doc = nlp(text)
    return " ".join([token.lemma_.lower() 
                     for token in doc 
                     if token.is_alpha and token.lemma_.lower() not in french_stop_words])

# Appliquer sur la colonne 'title'
data_fr['title_clean'] = data_fr['title'].apply(clean_title_fr)

C:\Users\charl\AppData\Local\Temp\ipykernel_12036\4166352686.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fr['title_clean'] = data_fr['title'].apply(clean_title_fr)


In [47]:
# Construction de la matrice TF-IDF pour les articles en français en utilisant seulement les titres
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_fr = TfidfVectorizer(
    vocabulary=vocab_fr,  # on utilise le vocabulaire extrait précédemment
    max_df=0.8,                 # optionnel, mais sécuritaire
    min_df=2,                   # optionnel, mais sécuritaire
    ngram_range=(1, 2)          # unigrams et bigrams
)

# Construction de la matrice TF-IDF
tfidf_matrix_fr = vectorizer_fr.fit_transform(data_fr['title_clean'])
print("TF-IDF matrix shape (fr):", tfidf_matrix_fr.shape)


TF-IDF matrix shape (fr): (1133, 1837)


In [48]:
# Application du K-means 
# En regardant les résultats du challenge publiés, on constate que le nombre de clusters utilisé est 10
from sklearn.cluster import KMeans
k = 10
kmeans = KMeans(n_clusters=k, random_state=42)
data_fr["cluster"] = kmeans.fit_predict(tfidf_matrix_fr)

c:\Users\charl\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\charl\AppData\Local\Temp\ipykernel_12036\462268148.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fr["cluster"] = kmeans.fit_predict(tfidf_matrix_fr)


In [49]:
terms = vectorizer_fr.get_feature_names_out()

In [50]:
# Chaque cluster a un centroïde (un vecteur) qui est la moyenne des vecteurs TF-IDF de tous les documents dans ce cluster.
# Chaque dimension du vecteur correspond à un mot de ton vocabulaire.
# Plus la valeur dans le centroïde pour un mot est grande, plus ce mot est représentatif des documents du cluster.
# Le code prend les n mots avec les plus grandes valeurs dans le centroïde → ce sont ceux qui définissent le cluster.
# Du coup le premier mot est le plus proche du thème du cluster
import numpy as np

def get_top_words_per_cluster(model, terms, n_words=10):
    for i, centroid in enumerate(model.cluster_centers_):
        top_indices = centroid.argsort()[-n_words:][::-1]
        top_terms = [terms[ind] for ind in top_indices]
        print(f"Cluster {i} : {', '.join(top_terms)}")
get_top_words_per_cluster(kmeans, terms)

Cluster 0 : ontologie, alignement, construction, sémantique, texte, guider, entrer, owl, dédier, relation
Cluster 1 : donnée, fouille, flux, base, entrepôt, visualisation, apprentissage, multidimensionnel, textuel, ensemble
Cluster 2 : motif, séquentiel, extraction, fréquent, découverte, donnée, contrainte, attribuer, graduel, base
Cluster 3 : règle, association, extraction, génération, recherche, classification, comparaison, exception, flou, entrer
Cluster 4 : réseau, apprentissage, sémantique, détection, graphe, social, recherche, information, mesure, clustering
Cluster 5 : séquence, temporel, série, spatio, événement, motif, extraction, classification, fréquent, flux
Cluster 6 : connaissance, extraction, gestion, base, donnée, compétence, information, acquisition, texte, expert
Cluster 7 : document, arbre, xml, décision, annotation, structure, recherche, classification, évaluation, classement
Cluster 8 : classification, superviser, non, donnée, sélection, image, automatique, visuali

In [51]:
for i in range(k):
    print(f"\n### Cluster {i}")
    print(data_fr[data_fr["cluster"] == i]["title"].head(3).values)


### Cluster 0
["Contraintes prescriptives compatibles avec OWL2-ER pour évaluer la complétude d'ontologies"
 "L'ontologie OntoBiotope pour l'étude de la biodiversité microbienne"
 "Vers une instance française de NELL : chaîne TLN multilingue et modélisation d'ontologie"]

### Cluster 1
['Apport de la fouille de données pour la prévention du risque suicidaire'
 'Interrogation de données structurellement hétérogènes dans les bases de données orientées documents'
 "L'exploitation de données contextuelles pour la recommandation d'hôtels"]

### Cluster 2
["Découverte de motifs graduels partiellement ordonnés : application aux données d'expériences scientifiques"
 'Echantillonnage de motifs séquentiels sous contrainte sur la norme'
 'Fouille de Motifs Graduels Fermés Fréquents Sous Contrainte de la Temporalité']

### Cluster 3
["Définir les catégories de DBpédia avec des règles d'associations et des redescriptions"
 "Sélection ciblée des descripteurs visuels pour la recherche d'images: une 

In [52]:
# Affichons le nombre d'articles par cluster
print(data_fr['cluster'].value_counts())

cluster
4    453
1    175
6    112
8     94
3     59
0     55
5     51
2     48
7     46
9     40
Name: count, dtype: int64


In [53]:
data_fr = data_fr.drop(['cluster', 'title_clean'], axis=1)
data_fr.to_csv("clustered_articles_fr.csv", index=False)

# Traitement de l'anglais

In [54]:
# Récupération du vocabulaire français à partir des titres des articles en anglais
# je veux le sauvgarder dans une liste 
vocab_en = set()
for title in data_en['title']:
    for word in title.split():
        vocab_en.add(word.lower())

print (len(vocab_en))
print ((vocab_en))

550
{'grouping', 'privacy:', 'broad', 'inférence,', 'sampling', 'media', 'sites', 'enhanced', 'time', 'view', 'geometry', 'have', 'reveal?', 'tom:', 'data:', 'sequences', 'anomalies', 'their', 'interactions', 'discovery', 'sentence-phrase-based', 'mining:', 'capture', 'reasoning', 'rewriting', 'limit', 'materialized', 'outlier', 'density', 'hadoop', 'smart', 'boost1', 'approximation', 'its', 'platform', 'genome-wide', 'entropy', 'event', 'using', 'rule', 'filling', 'memory', 'spaces', 'evaluating', 'compendium:', 'total', 'apps', 'web', 'handling', 'initiate', 'ssc', 'logic', 'simplified', 'classifying', 'user', 'literary', 'pls', 'knowledge', 'composite', 'multi-label', 'multi-classifier', 'database:', 'communities', 'entropic-genetic', 'framework', 'correction', 'privacy', 'presence', 'driven', 'algorithms', 'activity', 'combinaison', 'semi-interactif', 'visualization', 'as', 'dataset', 'leaning', 'ultrametricity', 'clustering', 'egc', 'rdf', 'incremental', 'relationship', 'mobile', 

In [55]:
# Lemmatisation

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def lemmatiser_anglais(texte):
    doc = nlp(str(texte).lower())
    # On garde le lemme si ce n'est pas une ponctuation ou un stop word
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

data_en['title_lemmatize'] = data_en['title'].apply(lemmatiser_anglais)

C:\Users\charl\AppData\Local\Temp\ipykernel_12036\174525150.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_en['title_lemmatize'] = data_en['title'].apply(lemmatiser_anglais)


In [56]:
# Affichage après lemmatisation
vocab_en = set()
for title in data_en['title_lemmatize']:
    for word in title.split():
        vocab_en.add(word.lower())

print (len(vocab_en))
print ((vocab_en))

458
{'visualisation', 'grouping', 'tree', 'broad', 'inférence', 'database', 'time', 'view', 'geometry', 'order', 'k', 'line', 'discovery', 'derive', 'reasoning', 'capture', 'limit', 'outlier', 'enhance', 'density', 'hadoop', 'smart', 'boost1', 'approximation', 'platform', 'open', 'base', 'entropy', 'event', 'rule', 'geo', 'memory', 'multi', 'hint', 'distribution', 'compact', 'total', 'strategy', 'web', 'initiate', 'ssc', 'logic', 'simplified', 'tweet', 'detect', 'evaluate', 'criterion', 'user', 'literary', 'moi', 'pls', 'distribute', 'analytic', 'knowledge', 'composite', 'fill', 'framework', 'correction', 'privacy', 'answer', 'presence', 'extract', 'activity', 'combinaison', 'visualization', 'dataset', 'ultrametricity', 'clustering', 'egc', 'datum', 'rdf', 'incremental', 'relationship', 'mobile', 'fairness', 'statistical', 'modeling', 'guide', 'fisher', 'deep', 'shoot', 'range', 'khiop', 'value', 'compendium', 'topology', 'online', 'logiciel', 'hierarchic', 'training', 'link', 'confere

In [57]:
# Désambiguïsation (regroupement de synonymes)

def reduire_synonymes(mot):
    # On cherche les synonymes du mot
    if len(str(mot)) <= 3:
        return mot
    synsets = wordnet.synsets(mot)
    if synsets:
        # On récupère le nom du premier concept (le plus commun)
        # ex: 'big' et 'large' pourraient tous deux renvoyer 'large'
        return synsets[0].lemmas()[0].name()
    return mot

def traiter_synonymes(texte_lemmatize):
    mots = str(texte_lemmatize).split()
    # Remplace chaque mot par son synonyme pivot
    mots_unifies = [reduire_synonymes(m) for m in mots]
    return " ".join(mots_unifies)

data_en['title_clear'] = data_en['title_lemmatize'].apply(traiter_synonymes)


C:\Users\charl\AppData\Local\Temp\ipykernel_12036\1867460168.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_en['title_clear'] = data_en['title_lemmatize'].apply(traiter_synonymes)


In [58]:
# Affichage après désambiguïsation (regroupement de synonymes)
vocab_en = set()
for title in data_en['title_clear']:
    for word in title.split():
        vocab_en.add(word.lower())

print (len(vocab_en))
print ((vocab_en))

442
{'tree', 'broad', 'inférence', 'database', 'time', 'geometry', 'order', 'k', 'line', 'discovery', 'inform', 'capture', 'reasoning', 'limit', 'outlier', 'property', 'enhance', 'density', 'hadoop', 'smart', 'boost1', 'platform', 'bibliographic', 'open', 'base', 'categorization', 'event', 'rule', 'geo', 'memory', 'multi', 'hint', 'distribution', 'compact', 'web', 'ssc', 'logic', 'tweet', 'detect', 'user', 'literary', 'moi', 'pls', 'distribute', 'analytic', 'homo', 'automatic_rifle', 'fill', 'correction', 'privacy', 'answer', 'presence', 'activity', 'combinaison', 'tilt', 'dataset', 'ultrametricity', 'visual_image', 'egc', 'datum', 'stopping_point', 'rdf', 'incremental', 'relationship', 'mobile', 'fairness', 'statistical', 'modeling', 'deep', 'shoot', 'bunch', 'procedure', 'prosecute', 'khiop', 'value', 'designation', 'topology', 'familial', 'logiciel', 'hanker', 'training', 'link', 'conference', 'year', 'svm', 'gene', 'interactif', 'random', 'regulate', 'spatial', 'future', 'microblog

In [59]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     ------ --------------------------------- 2.1/12.8 MB 7.8 MB/s eta 0:00:02
     ----------- ---------------------------- 3.7/12.8 MB 7.8 MB/s eta 0:00:02
     ---------------- ----------------------- 5.2/12.8 MB 8.0 MB/s eta 0:00:01
     ---------------------- ----------------- 7.1/12.8 MB 8.1 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 7.9 MB/s eta 0:00:01
     -------------------------------- ------- 10.5/12.8 MB 8.1 MB/s eta 0:00:01
     -------------------------------------- - 12.3/12.8 MB 8.0 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 7.6 MB/s  0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [60]:
# Construction de la matrice TF-IDF pour les articles en français en utilisant seulement les titres
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_en = TfidfVectorizer(
    vocabulary=vocab_en,  # on utilise le vocabulaire extrait précédemment
    max_df=0.8,                 # optionnel, mais sécuritaire
    min_df=2,                   # optionnel, mais sécuritaire
    ngram_range=(1, 2)          # unigrams et bigrams
)

# Construction de la matrice TF-IDF
tfidf_matrix_en = vectorizer_en.fit_transform(data_en['title_clear'])
print("TF-IDF matrix shape (en):", tfidf_matrix_en.shape)

TF-IDF matrix shape (en): (122, 442)


In [61]:
inertia = []
K = range(2, 10)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(tfidf_matrix_en)
    inertia.append(kmeans.inertia_)


c:\Users\charl\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [62]:
terms = vectorizer_en.get_feature_names_out()

In [63]:
import numpy as np

def get_top_words_per_cluster(model, terms, n_words=10):
    for i, centroid in enumerate(model.cluster_centers_):
        top_indices = centroid.argsort()[-n_words:][::-1]
        top_terms = [terms[ind] for ind in top_indices]
        print(f"Cluster {i} : {', '.join(top_terms)}")
get_top_words_per_cluster(kmeans, terms)

Cluster 0 : user, evaluation, retrieval, microblogge, colloquial, interaction, site, passage, open, ranking
Cluster 1 : network, model, ontology, learning, property, community, sociable, graph, machine, mine
Cluster 2 : extraction, information, semantic, text, complex, compact, kernel, system, intercede, processing
Cluster 3 : rule, discovery, association, set, base, database, stopping_point, action, relational, representative
Cluster 4 : stream, detect, data, anomaly, line, bunch, datum, detection, academician, plagiarism
Cluster 5 : datum, mining, constraint, big, categorization, scheduling, challenge, fusion, privacy, learn
Cluster 6 : analytic, ocular, problem, base, challenge, system, solve, bibliographic, keep, application
Cluster 7 : detection, outlier, collaborative, invasion, mobile, automaton, autonomous, small, antipattern, experiment
Cluster 8 : bunch, familial, heuristic, algorithm, entropic, base, network, visuel, interactif, ssc


In [64]:
# Application du K-means 
# En regardant les résultats du challenge publiés, on constate que le nombre de clusters utilisé est 10
k = 9
kmeans = KMeans(n_clusters=k, random_state=42)
data_en["cluster"] = kmeans.fit_predict(tfidf_matrix_en)

C:\Users\charl\AppData\Local\Temp\ipykernel_12036\1160871002.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_en["cluster"] = kmeans.fit_predict(tfidf_matrix_en)


In [65]:
for i in range(k):
    print(f"\n### Cluster {i}")
    print(data_en[data_en["cluster"] == i]["title"].values)


### Cluster 0
['Enhanced user-user collaborative filtering recommendation algorithm based on semantic ratings'
 '“Engage moi”: From retrieval effectiveness, user satisfaction to user engagement'
 'A Relevant Passage Retrieval and Re-ranking Approach for Open-Domain Question Answering'
 'Using Social Conversational Context For Detecting Users Interactions on Microblogging Sites'
 'User Evaluation: Why?']

### Cluster 1
['Big Data for understanding human dynamics: the power of networks'
 'Community structure in complex networks'
 'Long-range influences in (social) networks'
 'Reframing for Non-Linear Dataset Shift'
 'Temporal hints in the cultural heritage discourse: what can an ontology of time as it is worded reveal?'
 'Machine Learning Based Classification of Android Apps through Text Features'
 'Machine Learning for the Semantic Web: filling the gaps in Ontology Mining'
 'Analyse exploratoire par k-Coclustering avec Khiops CoViz'
 'TOM: A library for topic modeling and browsing'
 'A

In [66]:
# 1. Compter le nombre de titres par cluster
counts = data_en['cluster'].value_counts().sort_index()

# 2. Affichage propre
print("Nombre de documents par cluster :")
print("-" * 30)
for cluster_id, count in counts.items():
    if cluster_id == -1:
        print(f"Bruit (Noise -1) : {count} documents")
    else:
        print(f"Cluster {cluster_id}      : {count} documents")

Nombre de documents par cluster :
------------------------------
Cluster 0      : 5 documents
Cluster 1      : 36 documents
Cluster 2      : 13 documents
Cluster 3      : 15 documents
Cluster 4      : 8 documents
Cluster 5      : 24 documents
Cluster 6      : 7 documents
Cluster 7      : 4 documents
Cluster 8      : 10 documents


In [ ]:
# Association des thèmes
mapping_clusters = {
    0: "Recherche d’information et recommandation", 
    1: "Analyse de graphes, réseaux complexes et ontologies",
    2: "Extraction de connaissances et Web sémantique",
    3: "Règles d’association et fouille de données relationnelles",
    4: "Gestion de flux de données et détection d’anomalies",
    5: "Apprentissage automatique et méthodologies data mining",
    6: "Visualisation de données et analyse décisionnelle",
    7: "Détection de motifs et sécurité (Intrusion/Outliers)",
    8: "Algorithmes de clustering et partitionnement"
}

data_en['theme'] = data_en['cluster'].map(mapping_clusters)

for i in range(k):
    theme_nom = mapping_clusters.get(i, "Thème non défini")
    print(f"\n### Cluster {i} : {theme_nom}")
    print(data_en[data_en["cluster"] == i]["title"].values)


### Cluster 0 : Extraction d’information et analyse documentaire web
['Enhanced user-user collaborative filtering recommendation algorithm based on semantic ratings'
 '“Engage moi”: From retrieval effectiveness, user satisfaction to user engagement'
 'A Relevant Passage Retrieval and Re-ranking Approach for Open-Domain Question Answering'
 'Using Social Conversational Context For Detecting Users Interactions on Microblogging Sites'
 'User Evaluation: Why?']

### Cluster 1 : Data mining et gestion de flux de données
['Big Data for understanding human dynamics: the power of networks'
 'Community structure in complex networks'
 'Long-range influences in (social) networks'
 'Reframing for Non-Linear Dataset Shift'
 'Temporal hints in the cultural heritage discourse: what can an ontology of time as it is worded reveal?'
 'Machine Learning Based Classification of Android Apps through Text Features'
 'Machine Learning for the Semantic Web: filling the gaps in Ontology Mining'
 'Analyse explo

C:\Users\charl\AppData\Local\Temp\ipykernel_12036\2263000877.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_en['theme'] = data_en['cluster'].map(mapping_clusters)


In [37]:
# export csv
data_en = data_en.drop(['cluster', 'title_lemmatize' , 'title_clear'], axis=1)
data_en.to_csv("clustered_articles_en.csv", index=False)

# Merge des deux csv

In [ ]:
# Chargement
data_fr = pd.read_csv('clustered_articles_fr.csv', quotechar='"', skipinitialspace=True)
data_en = pd.read_csv('clustered_articles_en.csv', quotechar='"', skipinitialspace=True)

# Fusionner
df_final = pd.concat([data_fr, data_en], ignore_index=True)

# Sauvegarder le résultat
df_final.to_csv('articles.csv', index=False)